In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
data = pd.read_csv('/content/heart.csv')
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [3]:
# Set random seeds for reproducibility
seed_value = 42
tf.random.set_seed(seed_value)
np.random.seed(seed_value)

In [4]:

# Split the dataset
X = data.drop('target', axis=1)
y = data['target']

In [5]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=2)

In [6]:
# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
#  ANN model with fixed seeds
ANN = keras.Sequential([
    layers.Input(shape=(13,)),
    layers.Dense(64, activation='relu', kernel_initializer=tf.keras.initializers.GlorotUniform(seed=seed_value)),
    layers.Dropout(0.5),
    layers.Dense(32, activation='relu', kernel_initializer=tf.keras.initializers.GlorotUniform(seed=seed_value)),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.GlorotUniform(seed=seed_value))
])




In [8]:
# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Compile the model
ANN.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [9]:
# Train the model with early stopping
ANN.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.1, callbacks=[early_stopping])

# Make predictions
y_pred = ANN.predict(X_test)
y_pred = (y_pred > 0.5)  # Convert probabilities to binary values

# Calculate evaluation metrics
accuracy_ANN = accuracy_score(y_test, y_pred)
precision_ANN = precision_score(y_test, y_pred)
recall_ANN = recall_score(y_test, y_pred)
f1_ANN = f1_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)

from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score


y_pred_prob_ANN = ANN.predict(X_test).ravel()
auc_roc_ANN = roc_auc_score(y_test, y_pred_prob_ANN)
print(f'ANN AUC: {auc_roc_ANN}')


print(f'Accuracy: {accuracy_ANN:.2f}')
print(f'Precision: {precision_ANN:.2f}')
print(f'Recall: {recall_ANN:.2f}')
print(f'F1 Score: {f1_ANN:.2f}')
print('Confusion Matrix:')
print(confusion)

Epoch 1/100
26/26 [==============================] - 5s 24ms/step - loss: 0.7757 - accuracy: 0.4789 - val_loss: 0.6135 - val_accuracy: 0.7742
Epoch 2/100
26/26 [==============================] - 0s 9ms/step - loss: 0.6444 - accuracy: 0.5826 - val_loss: 0.5075 - val_accuracy: 0.8925
Epoch 3/100
26/26 [==============================] - 0s 4ms/step - loss: 0.5731 - accuracy: 0.6888 - val_loss: 0.4349 - val_accuracy: 0.9140
Epoch 4/100
26/26 [==============================] - 0s 4ms/step - loss: 0.5306 - accuracy: 0.7491 - val_loss: 0.3869 - val_accuracy: 0.9247
Epoch 5/100
26/26 [==============================] - 0s 4ms/step - loss: 0.5007 - accuracy: 0.7660 - val_loss: 0.3526 - val_accuracy: 0.9247
Epoch 6/100
26/26 [==============================] - 0s 4ms/step - loss: 0.4602 - accuracy: 0.7853 - val_loss: 0.3259 - val_accuracy: 0.9032
Epoch 7/100
26/26 [==============================] - 0s 4ms/step - loss: 0.4410 - accuracy: 0.8022 - val_loss: 0.3118 - val_accuracy: 0.9032
Epoch 8/100
